In [35]:
import pandas as pd
import os
import arcpy

os.environ["CRYPTOGRAPHY_OPENSSL_NO_LEGACY"] = "yes"

from arcgis.features import GeoAccessor, GeoSeriesAccessor


In [36]:
arcpy.env.overwriteOutput = True

poligono = 'sector'
numero_areas = 4

arcpy.env.workspace = r"C:\docsProyectos\8.GESTVALT\1.GIS\PRO\GESVALT\GESVALT.gdb"

In [37]:
for capa in arcpy.ListFeatureClasses():
    if capa == poligono:
        arcpy.management.SubdividePolygon(in_polygons = capa, 
            out_feature_class = 'sector_subdividido', 
            method = 'NUMBER_OF_EQUAL_PARTS', 
            num_areas = numero_areas, 
            subdivision_type = 'STACKED_BLOCKS')

In [38]:
df_sectores_subdivididos = pd.DataFrame.spatial.from_featureclass(os.path.join(arcpy.env.workspace, 'sector_subdividido'))

df_sectores_subdivididos['cantidad_sectores'] = (df_sectores_subdivididos.groupby('catastro').cumcount()+1).astype(str)

df_sectores_subdivididos['id_sector'] = df_sectores_subdivididos['catastro'] + '_' + df_sectores_subdivididos['cantidad_sectores']

est_columnas_sectores_subdivididos = ['id_sector','catastro','priorizacion','SHAPE']

df_sectores_subdivididos = df_sectores_subdivididos[est_columnas_sectores_subdivididos]

df_sectores_subdivididos.spatial.to_featureclass(location=os.path.join(arcpy.env.workspace, 'sector_salida_grafica'))

'C:\\docsProyectos\\8.GESTVALT\\1.GIS\\PRO\\GESVALT\\GESVALT.gdb\\sector_salida_grafica'

In [39]:
memoria_centroide_ptomuestra = "in_memory/centroide_ptoMuestra"
memoria_joinspatial_sectorMzn = "in_memory/joinspatial_sectorMzn"
memoria_joinspatial_sectorMznBarrios = "in_memory/joinspatial_sectorMznBarrios"
memoria_ptoMuestra_w_Sector ="in_memory/ptoMuestra_w_Sector"

for capa in arcpy.ListFeatureClasses():
    if capa == 'poligonos_puntos_muestra':
        arcpy.FeatureToPoint_management(in_features=capa, out_feature_class=memoria_centroide_ptomuestra, point_location='INSIDE')

        arcpy.SpatialJoin_analysis(target_features=memoria_centroide_ptomuestra,
        join_features='sector_salida_grafica',
        out_feature_class=memoria_joinspatial_sectorMzn,
        join_type = 'KEEP_COMMON',
        match_option='INTERSECT'
        )

        arcpy.SpatialJoin_analysis(target_features=memoria_joinspatial_sectorMzn,
        join_features='barrios',
        out_feature_class=memoria_joinspatial_sectorMznBarrios,
        join_type = 'KEEP_COMMON',
        match_option='INTERSECT'
        )

        arcpy.SpatialJoin_analysis(target_features=capa,
        join_features=memoria_joinspatial_sectorMznBarrios,
        out_feature_class=memoria_ptoMuestra_w_Sector,
        join_type = 'KEEP_COMMON',
        match_option='INTERSECT'
        )

        df_pto_muestra = pd.DataFrame.spatial.from_featureclass(memoria_ptoMuestra_w_Sector)

        est_columnas_pto_muestra = ['t_id','terrcnpn','id_sector','catastro','priorizacion','barrio','SHAPE']

        df_pto_muestra = df_pto_muestra[est_columnas_pto_muestra]

        df_pto_muestra.spatial.to_featureclass(location=os.path.join(arcpy.env.workspace,'poligono_pto_muestra_sector'))

In [40]:
query = "!OBJECTID!"

for capa in arcpy.ListFeatureClasses(wild_card='poligono_pto_muestra_sector'):
    arcpy.AddField_management(in_table=capa, field_name='id_punto_muestra', field_alias='ID Punto Muestra',field_type='SHORT')
    arcpy.CalculateField_management(in_table=capa, field='id_punto_muestra', expression=query, expression_type= 'PYTHON')